In [8]:
import pennylane as qml
from pennylane import numpy as np

from qiskit_ibm_runtime import QiskitRuntimeService
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


service = QiskitRuntimeService()
backend = service.backend("ibm_fez")

iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Classical baseline (local, fast)

clf_orig = LogisticRegression(max_iter=200, multi_class="auto")
clf_orig.fit(X_train_scaled, y_train)
y_pred_orig = clf_orig.predict(X_test_scaled)
acc_orig = accuracy_score(y_test, y_pred_orig)
print(f"Classical baseline (scaled) accuracy: {acc_orig*100:.2f}%")

# 4. Quantum device on IBM hardware

n_qubits = 4
shots = 256

dev = qml.device(
    "qiskit.remote",
    wires=n_qubits,
    backend=backend,
    shots=shots
)

# 5. Simple depth=1, entanglement=none embedding

def feature_map(x):
    for i in range(n_qubits):
        qml.RY(x[i], wires=i)

def param_layer(params):
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)

@qml.qnode(dev)
def quantum_embedding(x, params):
    feature_map(x)
    param_layer(params)
    # no entangling gates -> entanglement = "none"
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

def make_embeddings(X, params):
    embs = []
    for x in X:
        embs.append(quantum_embedding(x, params))
    return np.array(embs, dtype=float)

# 6. Random params (single seed) and embeddings

np.random.seed(0)
params = np.random.uniform(low=-0.5, high=0.5, size=(n_qubits,))

# Only embed a subset of training data
max_train_samples = min(20, len(X_train_scaled))
max_test_samples = len(X_test_scaled)

X_train_small = X_train_scaled[:max_train_samples]
y_train_small = y_train[:max_train_samples]
X_test_small = X_test_scaled[:max_test_samples]
y_test_small = y_test[:max_test_samples]

print(f"Running on {len(X_train_small)} train samples and {len(X_test_small)} test samples on IBM backend...")

X_train_emb = make_embeddings(X_train_small, params)
X_test_emb = make_embeddings(X_test_small, params)

print("Quantum embeddings (first 3 examples):")
print(X_train_emb[:3])

# 7. Classical classifier on quantum embeddings

clf_q = LogisticRegression(max_iter=200, multi_class="auto")
clf_q.fit(X_train_emb, y_train_small)
y_pred_q = clf_q.predict(X_test_emb)
acc_q = accuracy_score(y_test_small, y_pred_q)

print(f"Quantum (IBM) embedding accuracy on test set: {acc_q*100:.2f}%")


management.get:WARNING:2025-11-15 04:06:55,691: Loading default saved account
qiskit_runtime_service.__init__:WARNING:2025-11-15 04:07:00,182: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: QC. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-15 04:07:00,183: Using instance: QC, plan: open
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Classical baseline (scaled) accuracy: 86.67%


/usr/local/python/3.12.1/lib/python3.12/site-packages/pennylane/devices/device_api.py:193: PennyLaneDeprecationWarning: Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.
  warnings.warn(


Running on 20 train samples and 15 test samples on IBM backend...


KeyboardInterrupt: 